In [26]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
import pickle

In [27]:
## Load the dataset
data = pd.read_csv('Churn_Modelling.csv')
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [28]:
## Preprocess the dataset
# Drop unnecessary columns
data = data.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1)
data

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...
9995,771,France,Male,39,5,0.00,2,1,0,96270.64,0
9996,516,France,Male,35,10,57369.61,1,1,1,101699.77,0
9997,709,France,Female,36,7,0.00,1,0,1,42085.58,1
9998,772,Germany,Male,42,3,75075.31,2,1,0,92888.52,1


In [29]:
## Encode categorical variables
label_encoder_gender = LabelEncoder() ## since gender is a categorical variable
data['Gender'] = label_encoder_gender.fit_transform(data['Gender'])
data

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...
9995,771,France,1,39,5,0.00,2,1,0,96270.64,0
9996,516,France,1,35,10,57369.61,1,1,1,101699.77,0
9997,709,France,0,36,7,0.00,1,0,1,42085.58,1
9998,772,Germany,1,42,3,75075.31,2,1,0,92888.52,1


In [30]:
## One hot encode the 'Geography' column. Reason for one-hot encoding is that 'Geography' is a categorical variable with more than two categories.
from sklearn.preprocessing import OneHotEncoder
one_hot_encoder_geo = OneHotEncoder() 
geography_encoded = one_hot_encoder_geo.fit_transform(data[['Geography']])
geography_encoded

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 10000 stored elements and shape (10000, 3)>

In [31]:
one_hot_encoder_geo.get_feature_names_out(['Geography'])

array(['Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype=object)

In [32]:
geography_encoded_df = pd.DataFrame(geography_encoded.toarray(), columns=one_hot_encoder_geo.get_feature_names_out(['Geography']))
geography_encoded_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
...,...,...,...
9995,1.0,0.0,0.0
9996,1.0,0.0,0.0
9997,1.0,0.0,0.0
9998,0.0,1.0,0.0


In [33]:
## Combine the encoded geography data with the main dataset
data = pd.concat([data, geography_encoded_df], axis=1)      
data = data.drop(['Geography'], axis=1)  # Drop the original Geography column
data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [34]:
## Save the encoders
with open('label_encoder_gender.pkl', 'wb') as file:
    pickle.dump(label_encoder_gender, file)

with open('one_hot_encoder_geo.pkl', 'wb') as file:
    pickle.dump(one_hot_encoder_geo, file)

In [35]:
## Divide the dataset into features and target variable
X = data.drop('Exited', axis=1)
y = data['Exited']

## Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## Scale the features
scaler = StandardScaler() ## StandardScaler is used to standardize features by removing the mean and scaling to unit variance
X_train_scaled = scaler.fit_transform(X_train) 
X_test_scaled = scaler.transform(X_test)

In [36]:
X_train

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
9254,686,1,32,6,0.00,2,1,1,179093.26,1.0,0.0,0.0
1561,632,1,42,4,119624.60,2,1,1,195978.86,0.0,1.0,0.0
1670,559,1,24,3,114739.92,1,1,0,85891.02,0.0,0.0,1.0
6087,561,0,27,9,135637.00,1,1,0,153080.40,1.0,0.0,0.0
6669,517,1,56,9,142147.32,1,0,0,39488.04,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
5734,768,1,54,8,69712.74,1,1,1,69381.05,1.0,0.0,0.0
5191,682,0,58,1,0.00,1,1,1,706.50,1.0,0.0,0.0
5390,735,0,38,1,0.00,3,0,0,92220.12,1.0,0.0,0.0
860,667,1,43,8,190227.46,1,1,0,97508.04,1.0,0.0,0.0


In [37]:
## Save the scaler
with open('scaler.pkl', 'wb') as file:
    pickle.dump(scaler, file)

### ANN Implementation

In [38]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
import datetime

In [39]:
X_train_scaled.shape, X_test_scaled.shape, y_train.shape, y_test.shape

((8000, 12), (2000, 12), (8000,), (2000,))

In [40]:
## Build the ANN model
model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train_scaled.shape[1],)), ## Hidden Layer 1 connected to input layer
    Dense(32, activation='relu'), ## Hidden Layer 2 connected to Hidden Layer 1
    Dense(1, activation='sigmoid') ## Output layer
])

In [41]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                832       
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2945 (11.50 KB)
Trainable params: 2945 (11.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [45]:
import tensorflow
opt=tensorflow.keras.optimizers.Adam(learning_rate=0.001)  # Define the optimizer with a learning rate
loss='binary_crossentropy'  # Define the loss function for binary classification

In [46]:
## Compile the model
model.compile(optimizer=opt, loss=loss, metrics=['accuracy'])

In [47]:
## Set up the TensorBoard
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

In [50]:
## Set up early stopping to prevent overfitting
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

In [51]:
## Train the model
history = model.fit(
    X_train_scaled, y_train,
    validation_data=(X_test_scaled, y_test),
    epochs=100,
    callbacks=[tensorboard_callback, early_stopping]
)

Epoch 1/100
250/250 [==============================] - 0s 780us/step - loss: 0.3193 - accuracy: 0.8679 - val_loss: 0.3420 - val_accuracy: 0.8630
Epoch 2/100
250/250 [==============================] - 0s 713us/step - loss: 0.3189 - accuracy: 0.8664 - val_loss: 0.3437 - val_accuracy: 0.8590
Epoch 3/100
250/250 [==============================] - 0s 681us/step - loss: 0.3175 - accuracy: 0.8674 - val_loss: 0.3400 - val_accuracy: 0.8610
Epoch 4/100
250/250 [==============================] - 0s 672us/step - loss: 0.3171 - accuracy: 0.8681 - val_loss: 0.3443 - val_accuracy: 0.8645
Epoch 5/100
250/250 [==============================] - 0s 684us/step - loss: 0.3143 - accuracy: 0.8675 - val_loss: 0.3401 - val_accuracy: 0.8595
Epoch 6/100
250/250 [==============================] - 0s 677us/step - loss: 0.3124 - accuracy: 0.8698 - val_loss: 0.3446 - val_accuracy: 0.8630
Epoch 7/100
250/250 [==============================] - 0s 776us/step - loss: 0.3118 - accuracy: 0.8701 - val_loss: 0.3435 - val_ac

In [52]:
model.save('ann_model.h5')  # Save the trained model

/Users/a825765/Documents/me/NUS/Learn/Machine Learning/ANN/venv/lib/python3.11/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [55]:
## Load TensorBoard Extension
%load_ext tensorboard   

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [54]:
%tensorboard --logdir logs/fit

In [56]:
# Kill any existing TensorBoard processes
import subprocess
import signal
import os

try:
    # Kill any existing tensorboard processes
    subprocess.run(['pkill', '-f', 'tensorboard'], check=False)
    print("Killed existing TensorBoard processes")
except:
    print("No existing TensorBoard processes found")

# Wait a moment for processes to terminate
import time
time.sleep(2)

Killed existing TensorBoard processes


In [57]:
# Reload TensorBoard extension and start with fresh instance
%reload_ext tensorboard

# Start TensorBoard with explicit port and host
%tensorboard --logdir logs/fit --port 6007 --host localhost

In [58]:
# Alternative: Open TensorBoard in browser
import webbrowser
import time

print("TensorBoard should be accessible at: http://localhost:6007")
print("If the iframe above doesn't work, click this link or copy-paste the URL into your browser:")
print("http://localhost:6007")

# Optionally, automatically open in browser
# webbrowser.open('http://localhost:6007')

# Check if TensorBoard is running
import subprocess
try:
    result = subprocess.run(['lsof', '-i', ':6007'], capture_output=True, text=True)
    if result.stdout:
        print("\n✅ TensorBoard is running on port 6007")
    else:
        print("\n❌ TensorBoard might not be running on port 6007")
except:
    print("\nCouldn't check port status, but TensorBoard should be running")

TensorBoard should be accessible at: http://localhost:6007
If the iframe above doesn't work, click this link or copy-paste the URL into your browser:
http://localhost:6007

✅ TensorBoard is running on port 6007
